# Import Libs

In [162]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import SGD, Adam, lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import pandas as pd
import numpy  as np
from datetime import datetime
import time
import os

# Configuration

In [163]:
classes = [
    "angry", 
    "disgust", 
    "fear", 
    "happy", 
    "sad", 
    "surprise", 
    "neutral"
]

In [164]:
class Config:
    # dataset
    TRAIN_DS_PATH = './dataset/train.csv'
    VAL_DS_PATH = './dataset/val.csv'
    TEST_DS_PATH = './dataset/finaltest.csv'
    
    # images dir
    TRAIN_IMG_DIR = './dataset/train/'
    VAL_IMG_DIR  = './dataset/val/'
    TEST_IMG_DIR  = './dataset/finaltest/'
    
    # training hyperparams
    EPOCHS = 25
    LR = 1e-3
    BATCH_SIZE = 8
    NUM_WORKERS = 0
    SHUFFLE = True
    
    # saved model path
    MODEL_DIR = './model/'

In [165]:
cfg = Config()

In [166]:
# tensorboard writer
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
writer = SummaryWriter(f"runs/{current_time}")

# Dataset

In [167]:
# reference: https://github.com/omarsayed7/Deep-Emotion/blob/master/data_loaders.py
class EmotionDataset(Dataset):
    def __init__(self, csv_file, img_dir, datatype, transform):
        '''
        Pytorch Dataset class
        params:-
                 csv_file : the path of the csv file    (train, validation, test)
                 img_dir  : the directory of the images (train, validation, test)
                 datatype : data type for the dataset   (train, val, test)
                 transform: pytorch transformation over the data
        return :-
                 image, labels
        '''
        self.csv_file = pd.read_csv(csv_file)
        self.labels = self.csv_file['emotion']
        self.img_dir = img_dir
        self.datatype = datatype
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.img_dir + self.datatype + str(idx) + '.jpg').convert('RGB')
        labels = np.array(self.labels[idx])
        labels = torch.from_numpy(labels).long()

        if self.transform :
            img = self.transform(img)
        return img, labels

In [168]:
# tranformations
transformation= transforms.Compose([
    transforms.Resize(256),
#     transforms.Grayscale(3), # no need this since you .convert("RGB") at __getitem__
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [169]:
# create datasets
train_dataset = EmotionDataset(
    csv_file=cfg.TRAIN_DS_PATH, 
    img_dir=cfg.TRAIN_IMG_DIR,
    datatype='train',
    transform=transformation
)

validation_dataset = EmotionDataset(
    csv_file=cfg.VAL_DS_PATH, 
    img_dir=cfg.VAL_IMG_DIR,
    datatype='val',
    transform = transformation
)


test_dataset = EmotionDataset(
    csv_file=cfg.TEST_DS_PATH, 
    img_dir=cfg.TEST_IMG_DIR,
    datatype='finaltest',
    transform = transformation
)

In [170]:
# create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

# Helpers

In [171]:
def train_epoch(epoch, model, train_loader, criterion, optimizer, device, log_every_n_batches=100, writer=None):
    """Train the model for 1 epoch
    Args:
        epoch: Current training epoch
        model: nn.Module
        train_loader: train DataLoader
        criterion: callable loss function
        optimizer: pytorch optimizer
        device: torch.device
        log_every_n_batches: Metrics will log every n batches
        writer: Tensorboard SummaryWriter
    Returns
    -------
    Tuple[Float, Float]
        average train loss and average train accuracy for current epoch
    """
    
    train_losses = []
    train_corrects = []
    running_losses = 0.0
    running_corrects = 0.0
    total = 0.0
    model.train()

    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss & number of corrects for current batch
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        n_corrects = torch.sum(preds == labels.data).item()

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate runnings_loss, running_correct and total if log_every_n_batches > 0
        if log_every_n_batches:
            running_losses += loss.item()
            running_corrects += n_corrects
            total += labels.size(0)
            
            if batch_idx % log_every_n_batches == 0:
                print(f"loss: {loss.item():>7f}  [{batch_idx:>5d}/{len(train_loader):>5d}]")
                
                # write to tensorboard
                if writer:
                    writer.add_scalar("train_loss", 
                                      running_losses/log_every_n_batches,
                                      global_step=epoch*len(train_loader)+batch_idx)
                    writer.add_scalar("train_accuracy", 
                                      running_corrects/total,
                                      global_step=epoch*len(train_loader)+batch_idx)
                
                # reset to zero values for next logging
                running_losses = 0.0
                running_corrects = 0.0
                total = 0.0
            
        # accumulate loss and n_corrects for current batch for later average metric calculation
        train_losses.append(loss.item())
        train_corrects.append(n_corrects)
        
    ave_train_loss = sum(train_losses)/len(train_losses)
    ave_train_accuracy = sum(train_corrects)/len(train_loader.dataset)      

    return ave_train_loss, ave_train_accuracy

In [172]:
def val_epoch(epoch, model, val_loader, criterion, device, log_every_n_batches=100, writer=None):
    """Validate the model for 1 epoch
    Args:
        epoch: Current validation epoch
        model: nn.Module
        val_loader: val DataLoader
        criterion: callable loss function
        device: torch.device
        log_every_n_batches: Metrics will log every n batches
        writer: Tensorboard SummaryWriter
    Returns
    -------
    Tuple[Float, Float]
        average val loss and average val accuracy for current epoch
    """

    val_losses = []
    val_corrects = []
    running_losses = 0.0
    running_corrects = 0.0
    total = 0.0
    model.eval()

    # Iterate over data
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # prediction
            outputs = model(inputs)

            # calculate loss & number of corrects for current batch
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            n_corrects = torch.sum(preds == labels.data).item()
            
            # calculate runnings_loss, running_correct and total if log_every_n_batches > 0
            if log_every_n_batches:
                running_losses += loss.item()
                running_corrects += n_corrects
                total += labels.size(0)

                if batch_idx % log_every_n_batches == 0:
                    if writer:
                        writer.add_scalar("val_loss", 
                                          running_losses/log_every_n_batches,
                                          global_step=epoch*len(val_loader)+batch_idx)
                        writer.add_scalar("val_accuracy", 
                                          running_corrects/total,
                                          global_step=epoch*len(val_loader)+batch_idx)

                    # reset to zero values for next logging
                    running_losses = 0.0
                    running_corrects = 0.0
                    total = 0.0
                
            # accumulate loss and n_corrects for current batch for later average metric calculation
            val_losses.append(loss.item())
            val_corrects.append(n_corrects)
            
        ave_val_loss = sum(val_losses)/len(val_losses)
        ave_val_accuracy = sum(val_corrects)/len(val_loader.dataset)

    return ave_val_loss, ave_val_accuracy

# Training and Evaluating Model

In [173]:
# load pretrained model
model = models.mobilenet.mobilenet_v2(pretrained=True)

In [174]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))

In [175]:
# transfer to cuda device if any
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using {device}")

Using cuda


In [176]:
# loss, optimizer and scheduler 
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=cfg.LR)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max')
# lambda1 = lambda epoch: 0.65 ** epoch
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

# optimizer = SGD(model.parameters(), lr=cfg.LR)
# scheduler = lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1,step_size_up=5,mode="exp_range",gamma=0.85)


In [177]:
# Start training and val loops

since = time.time()
for epoch in range(cfg.EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    
    # Train
    ave_train_loss, ave_train_acc = train_epoch(epoch, model, train_loader, criterion, optimizer, device, 100, writer)
    print(f'Average TrainLoss: {ave_train_loss:.4f} \tAverage TrainAcc: {ave_train_acc:.4f}')
    
    # Evaluation
    print("Evaluation:")
    ave_val_loss, ave_val_acc = val_epoch(epoch, model, val_loader, criterion, device, 100, writer)
    print(f'Average ValLoss: {ave_val_loss:.4f} \tAverage ValAcc: {ave_val_acc:.4f}')
    
    # log metrics at every epoch
    if writer:
        writer.add_scalar("epoch_train_loss", ave_train_loss, epoch)
        writer.add_scalar("epoch_train_accuracy", ave_train_acc, epoch)
        writer.add_scalar("epoch_val_loss", ave_val_loss, epoch)
        writer.add_scalar("epoch_val_accuracy", ave_val_acc, epoch)
    
    # schedule lr
#     scheduler.step(ave_val_acc)
#     scheduler.step()
    print("\n")
        
time_elapsed = time.time() - since
print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")

Epoch 1
-------------------------------
loss: 1.890211  [    0/ 3589]
loss: 2.228004  [  100/ 3589]
loss: 1.632966  [  200/ 3589]
loss: 1.728315  [  300/ 3589]
loss: 1.403689  [  400/ 3589]
loss: 2.431549  [  500/ 3589]
loss: 1.836406  [  600/ 3589]
loss: 1.691087  [  700/ 3589]
loss: 1.666762  [  800/ 3589]
loss: 2.127090  [  900/ 3589]
loss: 1.844030  [ 1000/ 3589]
loss: 1.641667  [ 1100/ 3589]
loss: 1.625050  [ 1200/ 3589]
loss: 1.859219  [ 1300/ 3589]
loss: 1.449499  [ 1400/ 3589]
loss: 1.405838  [ 1500/ 3589]
loss: 1.114962  [ 1600/ 3589]
loss: 1.857995  [ 1700/ 3589]
loss: 1.677276  [ 1800/ 3589]
loss: 1.234138  [ 1900/ 3589]
loss: 1.882250  [ 2000/ 3589]
loss: 1.860530  [ 2100/ 3589]
loss: 1.622442  [ 2200/ 3589]
loss: 2.033559  [ 2300/ 3589]
loss: 2.212957  [ 2400/ 3589]
loss: 1.302111  [ 2500/ 3589]
loss: 1.856043  [ 2600/ 3589]
loss: 2.079294  [ 2700/ 3589]
loss: 2.333923  [ 2800/ 3589]
loss: 1.435002  [ 2900/ 3589]
loss: 1.979373  [ 3000/ 3589]
loss: 1.456432  [ 3100/ 3589]


KeyboardInterrupt: 

# Evaluating on Unseen Test Data

In [ ]:
test_losses = []
test_corrects = []

model.eval()
# Iterate over data
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        test_losses.append(loss.item())
        test_corrects.append(torch.sum(preds == labels.data).item())

    ave_test_loss = sum(test_losses)/len(test_losses)
    ave_test_acc = sum(test_corrects)/len(test_loader.dataset)
    
print(f'Average TestLoss: {ave_test_loss:.4f} \tAverage TestAcc: {ave_test_acc:.4f}')

# Saving model

In [ ]:
# Save the model
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
model_name = f"model_{current_time}_epoch{cfg.EPOCHS}_lr{cfg.LR}_batch{cfg.BATCH_SIZE}_acc{ave_test_acc:.3f}.pt"
torch.save(model.state_dict(), cfg.MODEL_DIR + model_name)